### Import Dataset

In [0]:
import pandas as pd

In [2]:
movies = pd.read_csv('https://raw.githubusercontent.com/skwiliam/Simple_KNN_from_Scratch/master/movies.csv')
ratings = pd.read_csv('https://raw.githubusercontent.com/skwiliam/Simple_KNN_from_Scratch/master/ratings.csv')

display(movies.head())
display(ratings.head())

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,1,1,5.0,847117005
1,1,2,3.0,847642142
2,1,10,3.0,847641896
3,1,32,4.0,847642008
4,1,34,4.0,847641956


### Create Feature

#### One Hot Encoding User Rating for Each Movie

In [0]:
#merge two dataset on movieId
df = pd.merge(movies[['movieId', 'title']], ratings, on='movieId')

#make a pivot dataframe
pivot_df = df.pivot_table(index='title',columns='userId',values='rating').fillna(0)

#features_df.sample(5)

#### One Hot Encoding Movies Genres

In [6]:
#add genre column
features_df = pd.merge(pivot_df, movies[['title', 'genres']], on='title')
features_df['genres'] = features_df.genres.str.split('|')

#onehot encoding genre at features_df df
features_df.genres.apply(set)
for genres in set.union(*features_df.genres.apply(set)):
    features_df[genres] = features_df.apply(lambda x: int(x.genres.count(genres)), axis=1)
features_df.drop('genres', axis=1, inplace=True)
features_df.head()

,title,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,IMAX,Thriller,War,Adventure,Romance,Film-Noir,Documentary,Musical,Animation,Comedy,Mystery,Drama,Horror,Sci-Fi,(no genres listed),Children,Crime,Action,Western,Fantasy
0,"""Great Performances"" Cats (1998)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
3,'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


### Feature Normalization

In [7]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        if feature_name != 'title':
            max_value = df[feature_name].max()
            min_value = df[feature_name].min()
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

norm_feature = normalize(features_df)
norm_feature.sample(5)

,title,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,IMAX,Thriller,War,Adventure,Romance,Film-Noir,Documentary,Musical,Animation,Comedy,Mystery,Drama,Horror,Sci-Fi,(no genres listed),Children,Crime,Action,Western,Fantasy
8707,Wild Target (2010),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2504,Election (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.8,0.0,0.6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3496,"Hangover Part III, The (2013)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8741,Winnie the Pooh and the Blustery Day (1968),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3553,Harum Scarum (1965),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### K Nearest Neighbor Algorithm

#### Eucladian Distance

![Eucladian Distance](https://wikimedia.org/api/rest_v1/media/math/render/svg/795b967db2917cdde7c2da2d1ee327eb673276c0)



In [0]:
def distance(point_p, point_q):
    #Calculate distance using Eucladian distance
    squared_difference = 0
    #calculate squared_difference between point from 1 to n
    for i in range(len(point_p)):
        squared_difference += (point_p[i] - point_q[i]) ** 2
    final_distance = squared_difference ** 0.5
    return final_distance

#### Find k Nearest Neighbor

In [0]:
def classify(title, dataset, k):
    distances = []
    title_feature = dataset[title]
    #Looping through all points in the dataset
    for data in dataset:
        point = dataset[data]
        distance_to_point = distance(title_feature, point)
        #Adding the distance and point associated with that distance
        distances.append([distance_to_point, data])
    distances.sort()
    #Taking only the k closest points
    neighbors = distances[0:k+1]
    return neighbors

### Recommendation

#### Create Dataset Dictionary from DataFrame

In [10]:
dataset = norm_feature.set_index('title').T.to_dict('list')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


#### Make Recommendation

In [11]:
title = "Avengers, The (2012)"
make_recommendation = classify(title, dataset, 10)
for i in range(0, len(make_recommendation)):
    if i == 0:
        print('Recommendations for {0}:\n'.format(make_recommendation[0][1]))
    else:
        print('{0}: {1}, with distance of: {2}'.format(i, make_recommendation[i][1], make_recommendation[i][0]))

Recommendations for Avengers, The (2012):

1: Captain America: The Winter Soldier (2014), with distance of: 3.919734810883808
2: Star Trek Into Darkness (2013), with distance of: 3.930256172343848
3: Dark Knight Rises, The (2012), with distance of: 3.9820585283816197
4: Hunger Games: Catching Fire, The (2013), with distance of: 3.9890436367383186
5: Iron Man 3 (2013), with distance of: 4.016524509548332
6: Amazing Spider-Man, The (2012), with distance of: 4.1161230710223515
7: Mission: Impossible - Ghost Protocol (2011), with distance of: 4.157768276015036
8: Guardians of the Galaxy (2014), with distance of: 4.185822632543227
9: Gravity (2013), with distance of: 4.190097084823561
10: Transformers: Revenge of the Fallen (2009), with distance of: 4.198441803841862
